In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
!pip install pandas
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Looking in indexes: http://pypi.idea.edu.cn


In [7]:
import sys 
import argparse
import logging
import os
import pickle
import random
import torch
import json
import numpy as np
import pandas as pd
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,BertConfig,BertModel, AlbertConfig, AlbertModel,DebertaV2Config,DebertaV2Model,
                              RobertaConfig, RobertaModel, RobertaTokenizer)
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import math
from collections import defaultdict
from itertools import chain
from torch.optim import Optimizer
import torch
import warnings

In [8]:
label_smoothing = 0.75
num_class  = 250
num_landmark = 543
max_length = 256
# point_dim  = 1323
embed_dim  = 240
num_head   = 16
num_block  = 3
root_dir = '.'

LHAND = np.arange(468, 489).tolist() # 21
RHAND = np.arange(522, 543).tolist() # 21
POSE  = np.arange(489, 522).tolist() # 33
FACE  = np.arange(0,468).tolist()    #468

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
][::2]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
][::2]
NOSE=[
    1,2,98,327
]
SLIP = [
    78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    191, 80, 81, 82, 13, 312, 311, 310, 415,
]
SPOSE = (np.array([
    11,13,15,12,14,16,23,24,
])+489).tolist()

BODY = REYE+LEYE+NOSE+SLIP+SPOSE

def get_indexs(L):
    return sorted([i + j * len(L) for i in range(len(L)) for j in range(len(L)) if i>j])

DIST_INDEX = get_indexs(RHAND)

LIP_DIST_INDEX = get_indexs(SLIP)

POSE_DIST_INDEX = get_indexs(SPOSE)

EYE_DIST_INDEX = get_indexs(REYE)

NOSE_DIST_INDEX = get_indexs(NOSE)

HAND_START = [0,1,2,3,5,6,7,9,10,11,13,14,15,17,18,19,0,5,9,13,0]
HAND_END = [1,2,3,4,6,7,8,10,11,12,14,15,16,18,19,20,5,9,13,17,17]

point_dim = point_dim2 = len(LHAND+RHAND+REYE+LEYE+NOSE+SLIP+SPOSE)*2+len(LHAND+RHAND)*2+len(RHAND)+len(DIST_INDEX)*2+len(EYE_DIST_INDEX)*2+len(LIP_DIST_INDEX)+len(POSE_DIST_INDEX)

def pack_seq(
	seq,
):
	length = [min(len(s), max_length)  for s in seq]
	batch_size = len(seq)
	K = seq[0].shape[1]
	L = max(length)
	#print(length)

	x = torch.zeros((batch_size, L, point_dim))#.to(seq[0].device)
	x_mask = torch.zeros((batch_size, L))#.to(seq[0].device)
	for b in range(batch_size):
		l = length[b]
		x[b, :l] = seq[b][:l,:]
		x_mask[b, l:] = 1
	x_mask = (x_mask>0.5)

	return x, x_mask

def positional_encoding(length, embed_dim):
	dim = embed_dim//2
	position = np.arange(length)[:, np.newaxis]     # (seq, 1)
	dim = np.arange(dim)[np.newaxis, :]/dim   # (1, dim)
	angle = 1 / (10000**dim)         # (1, dim)
	angle = position * angle    # (pos, dim)
	pos_embed = np.concatenate(
		[np.sin(angle), np.cos(angle)],
		axis=-1
	)
	pos_embed = torch.from_numpy(pos_embed).float()
	return pos_embed

class FCLayer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, split=4):
        super().__init__()
        self.dim = embed_dim//4
        self.mlp = nn.ModuleList(
            [nn.Linear(embed_dim//4, hidden_dim//4) for i in range(split)]
        )
        self.split=split
    def forward(self, x):
        output = []
        for i in range(self.split):
            output.append(self.mlp[i](x[:,:,self.dim*i:self.dim*i+self.dim]))
        x = torch.cat(output, -1)
        return x

class TransformerBlock(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x, x_mask=None):
		x = x + self.attn((self.norm1(x)), x_mask)
		x = x + self.ffn((self.norm2(x)))
		return x
    
class MyMultiHeadAttention(nn.Module):
	def __init__(self,
			embed_dim,
			out_dim,
			qk_dim,
			v_dim,
			num_head,
		):
		super().__init__()
		self.embed_dim = embed_dim
		self.num_head  = num_head
		self.qk_dim = qk_dim
		self.v_dim  = v_dim

		self.q = nn.Linear(embed_dim, qk_dim*num_head)
		self.k = nn.Linear(embed_dim, qk_dim*num_head)
		self.v = nn.Linear(embed_dim, v_dim*num_head)

		self.out = nn.Linear(v_dim*num_head, out_dim)
		self.scale = 1/(qk_dim**0.5)

	#https://github.com/pytorch/pytorch/issues/40497
	def forward(self, x, x_mask):
		B,L,dim = x.shape
		#out, _ = self.mha(x,x,x, key_padding_mask=x_mask)
		num_head = self.num_head
		qk_dim = self.qk_dim
		v_dim = self.v_dim

		q = self.q(x)
		k = self.k(x)
		v = self.v(x)
		q = q.reshape(B, L, num_head, qk_dim).permute(0,2,1,3).contiguous()
		k = k.reshape(B, L, num_head, qk_dim).permute(0,2,3,1).contiguous()
		v = v.reshape(B, L, num_head, v_dim ).permute(0,2,1,3).contiguous()

		dot = torch.matmul(q, k) *self.scale  # H L L
		x_mask = x_mask.reshape(B,1,1,L).expand(-1,num_head,L,-1)
		#dot[x_mask]= -1e4
		dot.masked_fill_(x_mask, -1e4)
		attn = F.softmax(dot, -1)    # L L

		v = torch.matmul(attn, v)  # L H dim
		v = v.permute(0,2,1,3).reshape(B,L, v_dim*num_head).contiguous()
		out = self.out(v)

		return out
    

class FeedForward(nn.Module):
	def __init__(self, embed_dim, hidden_dim):
		super().__init__()
		self.mlp = nn.Sequential(
			nn.Linear(embed_dim, hidden_dim),
            # FCLayer(embed_dim, hidden_dim,4),
			nn.ReLU(inplace=True),
            # FCLayer(embed_dim, hidden_dim,4),
			nn.Linear(hidden_dim, embed_dim),
		)
	def forward(self, x):
		return self.mlp(x)

    

In [9]:
class InputNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.max_length = max_length 
  
    def forward(self, xyz):
        xyz = xyz - xyz[~torch.isnan(xyz)].mean(0,keepdim=True) #noramlisation to common maen
        xyz = xyz / xyz[~torch.isnan(xyz)].std(0, keepdim=True)

        LIP = [
            78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
            191, 80, 81, 82, 13, 312, 311, 310, 415,
        ]
        #LHAND = np.arange(468, 489).tolist()
        #RHAND = np.arange(522, 543).tolist()

        lip = xyz[:, LIP]
        lhand = xyz[:, 468:489]
        rhand = xyz[:, 522:543]
        xyz = torch.cat([  # (none, 82, 3)
            lip,
            lhand,
            rhand,
        ], 1)
        xyz[torch.isnan(xyz)] = 0
        x = xyz[:self.max_length]
        return x


In [10]:


#overwrite the model used in training ....
class MyMultiHeadAttention(nn.Module):
	def __init__(self,
			embed_dim,
			out_dim,
			qk_dim,
			v_dim,
			num_head,
		):
		super().__init__()
		self.embed_dim = embed_dim
		self.num_head  = num_head
		self.qk_dim = qk_dim
		self.v_dim  = v_dim

		self.q = nn.Linear(embed_dim, qk_dim*num_head)
		self.k = nn.Linear(embed_dim, qk_dim*num_head)
		self.v = nn.Linear(embed_dim, v_dim*num_head)

		self.out = nn.Linear(v_dim*num_head, out_dim)
		self.scale = 1/(qk_dim**0.5)

	#https://github.com/pytorch/pytorch/issues/40497
	def forward(self, x):
		B,L,dim = x.shape
		#out, _ = self.mha(x,x,x, key_padding_mask=x_mask)
		num_head = self.num_head
		qk_dim = self.qk_dim
		v_dim = self.v_dim

		q = self.q(x)
		k = self.k(x)
		v = self.v(x)
		q = q.reshape(B, L, num_head, qk_dim).permute(0,2,1,3).contiguous()
		k = k.reshape(B, L, num_head, qk_dim).permute(0,2,3,1).contiguous()
		v = v.reshape(B, L, num_head, v_dim ).permute(0,2,1,3).contiguous()

		dot = torch.matmul(q, k) *self.scale  # H L L
		#x_mask = x_mask.reshape(B,1,1,L).expand(-1,num_head,L,-1)
		#dot[x_mask]= -1e4
		#dot.masked_fill_(x_mask, -1e4)
		attn = F.softmax(dot, -1)    # L L

		v = torch.matmul(attn, v)  # L H dim
		v = v.permute(0,2,1,3).reshape(B,L, v_dim*num_head).contiguous()
		out = self.out(v)

		return out
    

# remove mask
class TransformerBlock2(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class TransformerBlock3(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=24, #embed_dim // num_head,
			v_dim=24, #embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class TransformerBlock(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class XEmbed(nn.Module):
    def __init__(self,):
        super().__init__()
        self.v = nn.Sequential(
 			nn.Linear(point_dim, embed_dim*2, bias=True),
 			nn.LayerNorm(embed_dim*2),
 			nn.ReLU(inplace=True),
 			nn.Linear(embed_dim*2, embed_dim, bias=True),
 			nn.LayerNorm(embed_dim),
 			nn.ReLU(inplace=True),
 		)
    def forward(self, x):
        v = self.v(x)
        x = v
        return x


class XEmbed2(nn.Module):
    def __init__(self,):
        super().__init__()
        self.v = nn.Sequential(
 			nn.Linear(point_dim, 480*2, bias=True),
 			nn.LayerNorm(480*2),
 			nn.ReLU(inplace=True),
 			nn.Linear(480*2, 480, bias=True),
 			nn.LayerNorm(480),
 			nn.ReLU(inplace=True),
 		)
    def forward(self, x):
        v = self.v(x)
        x = v
        return x

class SingleNet(nn.Module):

    def __init__(self, num_class=num_class):
        super().__init__()
        self.num_block = 3
        self.embed_dim = embed_dim
        self.num_head  = num_head
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed()

        self.encoder = nn.ModuleList([
            TransformerBlock(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,embed_dim)
        x = torch.cat([
                self.cls_embed.reshape(1,1,embed_dim),
                x
            ],1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)
        
        logit = self.logit(x.mean(1))
        return logit
    

def positional_encoding1(length, embed_dim):
	dim = embed_dim//2
	position = np.arange(length)[:, np.newaxis]     # (seq, 1)
	dim = np.arange(dim)[np.newaxis, :]/dim   # (1, dim)
	angle = 1 / (500**dim)         # (1, dim)
	angle = position * angle    # (pos, dim)
	pos_embed = np.concatenate(
		[np.sin(angle), np.cos(angle)],
		axis=-1
	)
	pos_embed = torch.from_numpy(pos_embed).float()
	return pos_embed

class SingleNet2(nn.Module):

    def __init__(self, max_length):
        super().__init__()
        self.num_block = 3
        self.embed_dim = 480
        self.num_head  = 16
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed2()

        self.encoder = nn.ModuleList([
            TransformerBlock2(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,self.embed_dim)
        x = torch.cat([
                self.cls_embed.reshape(1,1,self.embed_dim),
                x
            ],1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)

        logit = self.logit(x.mean(1))
        return logit
    
class SingleNet3(nn.Module):
    def __init__(self, max_length):
        super().__init__()
        self.num_block = 3
        self.embed_dim = 512
        self.num_head  = 32
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding1(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed()

        self.norm = nn.LayerNorm(self.embed_dim)

        self.encoder = nn.ModuleList([
            TransformerBlock3(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,self.embed_dim)
        x = self.norm(x)
        x = torch.cat([
                self.cls_embed.reshape(1,1,self.embed_dim),
                x
            ],1)
        #x = x.unsqueeze(1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)
        #x = F.dropout(x,p=0.4,training=False)

        logit = self.logit(x.mean(1))
        return logit

In [11]:
class FinalNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.m1 = SingleNet2(384)
        checkpoint_file1 = '/home/zhouyuanzhe/code5th/kaggleasl5thplacesolution/codelarge/ckpts/3layer_emb480_full/checkpoints/epoch_0.pth'

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        state_dict = torch.load(checkpoint_file1)['state_dict']
        for k, v in state_dict.items():
            new_state_dict[k.replace('module.','')] = v
        self.m1.load_state_dict(new_state_dict)


        self.m2 = SingleNet()
        checkpoint_file2 = '/home/zhouyuanzhe/code5th/kaggleasl5thplacesolution/codesmall/ckpts/3layer_emb240_full/checkpoints/epoch_0.pth'

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        state_dict = torch.load(checkpoint_file2)['state_dict']
        for k, v in state_dict.items():
            new_state_dict[k.replace('module.','')] = v
        self.m2.load_state_dict(new_state_dict)


    def forward(self, xyz):
        logit = F.softmax(self.m1(xyz), -1)
        logit = logit + F.softmax(self.m2(xyz), -1)
        return logit

In [12]:
final_net = FinalNet()
final_net.eval()
1

1

In [19]:
!pip install onnx
!pip install onnx_tf 
!pip install onnxsim
!pip install tensorflow

Looking in indexes: http://pypi.idea.edu.cn
Looking in indexes: http://pypi.idea.edu.cn
Looking in indexes: http://pypi.idea.edu.cn
Looking in indexes: http://pypi.idea.edu.cn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 4.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 452.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 640.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 4.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.5 MB/s eta 0:00:0000:0100:010m
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━

In [21]:
!pip install tensorflow_probability

Looking in indexes: http://pypi.idea.edu.cn
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/d9/2e/d1b4166e7c5235a745ce85763502d4c3e20486d6cd2a0517583817734d40/tensorflow_probability-0.19.0-py2.py3-none-any.whl (6.7 MB)
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/e3/03/cd776c4f224aabe06225c3661f79e1114dbe337506ae9039575eb06cc568/dm_tree-0.1.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (152 kB)


In [22]:
import onnx
import onnx_tf
from onnx_tf.backend import prepare
import tensorflow as tf
import onnxsim


/home/zhouyuanzhe/anaconda3/envs/base4kg/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Installing onnxruntime by `/home/zhouyuanzhe/anaconda3/envs/base4kg/bin/python -m pip install onnxruntime`, please 
wait for a moment..

Looking in indexes: http://pypi.idea.edu.cn
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/3c/6d/e17bcd234194606803ebf7223ef58a8fc69d8f6e9b2e1ecd698bb5096c1c/onnxruntime-1.14.1-cp38-cp38-manylinux_2_27_x86_64.whl (5.0 MB)
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/2d/49/a2d03101e2d28ad528968144831d506344418ef1cc04839acdbe185889c2/sympy-1.11.1-py3-none-any.whl (6.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 318.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 560.0 kB/s eta 0:00:00a 0:00:01
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl (536 kB)


In [23]:
max_length = 130 #1601

In [24]:
class InputNet(tf.keras.layers.Layer):
    def __init__(self, ):
        super(InputNet, self).__init__()
        self.lip = tf.constant(SLIP)
        self.lhand = tf.constant(LHAND)
        self.rhand = tf.constant(RHAND)
        self.pose = tf.constant(SPOSE)
        self.reye = tf.constant(REYE)
        self.leye = tf.constant(LEYE)
        self.nose = tf.constant(NOSE)
        self.hand_start = tf.constant(HAND_START)
        self.hand_end = tf.constant(HAND_END)
        self.max_length = max_length

    def call(self, xyz):
        #x = xyz
        xyz = xyz[:,:,:2]
        L = len(xyz)
        if L > self.max_length:

            i = (L-self.max_length)//2
            xyz = xyz[i:i + self.max_length]  # center


        not_nan_xyz =  tf.reshape(xyz[~tf.math.is_nan(xyz)], (-1,2))
        xyz -= tf.math.reduce_mean (not_nan_xyz, axis=0, keepdims=True)   # noramlisation to common maen
        xyz /= tf.reduce_mean(tf.math.reduce_std (not_nan_xyz, axis=0, keepdims=True))
        
        x = tf.concat([
            tf.gather(xyz, self.lhand, axis=1),
            tf.gather(xyz, self.rhand, axis=1),
            tf.gather(xyz, self.lip, axis=1),
            tf.gather(xyz, self.pose, axis=1),
            tf.gather(xyz, self.reye, axis=1),
            tf.gather(xyz, self.leye, axis=1),
            tf.gather(xyz, self.nose, axis=1),
        ],1)
        return  x

In [25]:
class FeatNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.DIST_INDEX = DIST_INDEX
        self.LIP_DIST_INDEX = LIP_DIST_INDEX
        self.POSE_DIST_INDEX = POSE_DIST_INDEX
        self.EYE_DIST_INDEX = EYE_DIST_INDEX
  
    def forward(self, xyz):
        lhand = xyz[:, 0:21,:]#21
        rhand = xyz[:, 21:42,:]#21
        lip   = xyz[:, 42:62,:]#20
        pose = xyz[:, 62:70,:]#8
        reye = xyz[:, 70:78,:]#16
        leye = xyz[:, 78:86,:]#16
        nose = xyz[:, 86:90,:]#4


        x = torch.cat([xyz[1:,:len(LHAND+RHAND),:]-xyz[:-1,:len(LHAND+RHAND),:],torch.zeros((1,len(LHAND+RHAND),2))],0)
        
        ld = lhand[:,:,:2].reshape(-1,len(LHAND),1,2)-lhand[:,:,:2].reshape(-1,1,len(LHAND),2)
        ld = torch.sqrt((ld**2).sum(-1))
        ld = ld.reshape(-1,len(LHAND)*len(LHAND))[:,DIST_INDEX]
        
        rd = rhand[:,:,:2].reshape(-1,len(LHAND),1,2)-rhand[:,:,:2].reshape(-1,1,len(LHAND),2)
        rd = torch.sqrt((rd**2).sum(-1))
        rd = rd.reshape(-1,len(LHAND)*len(LHAND))[:,DIST_INDEX]
        
        lipd = lip[:,:,:2].reshape(-1,len(SLIP),1,2)-lip[:,:,:2].reshape(-1,1,len(SLIP),2)
        lipd = torch.sqrt((lipd**2).sum(-1))
        lipd = lipd.reshape(-1,len(SLIP)*len(SLIP))[:,LIP_DIST_INDEX]
        
        posed = pose[:,:,:2].reshape(-1,len(SPOSE),1,2)-pose[:,:,:2].reshape(-1,1,len(SPOSE),2)
        posed = torch.sqrt((posed**2).sum(-1))
        posed = posed.reshape(-1,len(SPOSE)*len(SPOSE))[:,POSE_DIST_INDEX]
        
        reyed = reye[:,:,:2].reshape(-1,len(REYE),1,2)-reye[:,:,:2].reshape(-1,1,len(REYE),2)
        reyed = torch.sqrt((reyed**2).sum(-1))
        reyed = reyed.reshape(-1,len(REYE)*len(REYE))[:,EYE_DIST_INDEX]
        
        leyed = leye[:,:,:2].reshape(-1,len(LEYE),1,2)-leye[:,:,:2].reshape(-1,1,len(LEYE),2)
        leyed = torch.sqrt((leyed**2).sum(-1))
        leyed = leyed.reshape(-1,len(LEYE)*len(LEYE))[:,EYE_DIST_INDEX]

        dist_hand=torch.sqrt(((lhand-rhand)**2).sum(-1))

        xyz = torch.cat([xyz.reshape(-1,(len(LHAND+RHAND+REYE+LEYE+NOSE+SLIP+SPOSE))*2), 
                            x.reshape(-1,(len(LHAND+RHAND))*2),
                            ld,
                            rd,
                            lipd,
                            posed,
                            reyed,
                            leyed,
                            dist_hand,
                            ],1)
        
        xyz[torch.isnan(xyz)] = 0
            
        return xyz

In [26]:
feat_net  = FeatNet()
feat_net.eval()

FeatNet()

In [27]:
#pytorch to onnx to tflite
fold_dir = './'
if 1:
    
    name='model' 
    input_onnx_file   = f'{fold_dir}/{name}.input.onnx'
    single_onnx_file  = f'{fold_dir}/{name}.single.onnx' 
    input_tf_file    = f'{fold_dir}/input_tf'
    single_tf_file   = f'{fold_dir}/single_tf'
    tf_file     = f'{fold_dir}/tf'
    tflite_file = f'{fold_dir}/{name}.tflite'

    def run_convert_onnx(): 
        if 1:
            torch.onnx.export(
                feat_net,
                #torch.jit.script(input_net),
                #torch.jit.trace(input_net, torch.zeros(100,num_landmark,3)),          # model being run 
                torch.zeros((100,90,2)), # model input (or a tuple for multiple inputs)
                input_onnx_file,             # where to save the model (can be a file or file-like object)
                export_params = True,        # store the trained parameter weights inside the model file
                opset_version = 12,          # the ONNX version to export the model to
                do_constant_folding=True,    # whether to execute constant folding for optimization 
                input_names =  ['inputs'],    # the model's input names
                output_names = ['outputs'],   # the model's output names
                dynamic_axes={
                    'inputs': {0: 'length'},
                    'output': {0: 'length'},
                },
                #verbose = True,
            )
            torch.onnx.export(
                final_net,         
                #torch.jit.script(single_net),
                #torch.jit.trace(single_net, torch.zeros(max_length,82,3)),           

                torch.zeros((max_length,point_dim)), 
                single_onnx_file,             
                export_params = True,         
                opset_version = 12, 
                do_constant_folding=True,      
                input_names =  ['inputs'],     
                output_names = ['outputs'],  
                dynamic_axes={
                    'inputs': {0: 'length'},
                },
                #verbose = True,
            )
            print('torch.onnx.export() passed !!')

        if 1:
            for f in [input_onnx_file, single_onnx_file]:
                print(1)
                if f is None: continue
                model = onnx.load(f)
                onnx.checker.check_model(model)
                #model_simple, check = onnxsim.simplify(model)
                onnx.save(model, f)
            print('onnx simplify() passed !!')


    def run_convert_tflite():
        if 1:
            tf_rep = prepare(onnx.load(input_onnx_file))
            tf_rep.export_graph(input_tf_file) 
            tf_rep = prepare(onnx.load(single_onnx_file))
            tf_rep.export_graph(single_tf_file) 
            print('tf_rep.export_graph() passed !!')

        if 1:
            class TFModel(tf.Module):
                def __init__(self):
                    super(TFModel, self).__init__()
                    #self.input  = tf.saved_model.load(input_tf_file)
                    self.input = InputNet()
                    self.feat = tf.saved_model.load(input_tf_file)
                    self.single = tf.saved_model.load(single_tf_file)
                    self.input.trainable = False
                    # self.single.trainable = False

                @tf.function(input_signature=[
                    tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
                ])
                def call(self, input):
                    y = {}
                    x = self.input(input)
                    x = self.feat(**{'inputs': x})['outputs']
                    y['outputs'] = self.single(**{'inputs': x})['outputs']
                    
                    return y

            tfmodel = TFModel()
            tf.saved_model.save(tfmodel, tf_file, signatures={'serving_default': tfmodel.call})
            print('tf.saved_model() passed !!')

        if 1:
            converter = tf.lite.TFLiteConverter.from_saved_model(tf_file)
            # converter.target_spec.supported_ops = [
            #     tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
            #     tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
            # ]
            #converter.optimizations = [tf.lite.Optimize.DEFAULT]
            #converter.target_spec.supported_types = [tf.FLOAT16]
            #converter.allow_custom_ops = True
            #converter.experimental_new_converter = True 
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]
            tf_lite_model = converter.convert()
            with open(tflite_file, 'wb') as f:
                f.write(tf_lite_model)
            print('tflite convert() passed !!')
 
    run_convert_onnx()
    run_convert_tflite()

/home/zhouyuanzhe/anaconda3/envs/base4kg/lib/python3.8/site-packages/torch/onnx/utils.py:1294: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))


torch.onnx.export() passed !!
1
1
onnx simplify() passed !!


2023-05-05 13:03:33.761711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8550 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-05-05 13:03:33.763218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 7111 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2023-05-05 13:03:33.764601: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9600 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5
2023-05-05 13:03:33.765907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 7995 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

INFO:tensorflow:Assets written to: .//input_tf/assets


INFO:tensorflow:Assets written to: .//input_tf/assets
2023-05-05 13:03:40.547153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,979]
	 [[{{node inputs}}]]
2023-05-05 13:03:40.547240: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '5223' with dtype float and shape [480]
	 [[{{node 5223}}]]
2023-05-05 13:03:40.547290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '5269' with dtype float and shape 

...

INFO:tensorflow:Assets written to: .//tf/assets


INFO:tensorflow:Assets written to: .//tf/assets


tf.saved_model() passed !!


2023-05-05 13:04:15.491127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype float and shape [?,543,3]
	 [[{{node serving_default_inputs}}]]
2023-05-05 13:04:42.170232: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-05 13:04:42.170276: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-05 13:04:42.170919: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: .//tf
2023-05-05 13:04:42.197648: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-05 13:04:42.197679: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: .//tf
2023-05-05 13:04:42.274297: I tensorflow/com

tflite convert() passed !!


In [28]:
!zip submission.zip  'model.tflite'
# !unzip -o submission.zip
# !ls

  adding: model.tflite (deflated 11%)


In [29]:
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / 543)
    data = data.values.reshape(n_frames, 543, len(data_columns))
    return data.astype(np.float32)

def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic


train = pd.read_csv(f"./train.csv")
label_index = read_dict(f"./sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])

In [32]:
!pip install tflite_runtime

Looking in indexes: http://pypi.idea.edu.cn
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/5f/1c/f7711255af47139f8d47d05f99de49dadd6f882b9cd81fce282872455206/tflite_runtime-2.11.0-cp38-cp38-manylinux2014_x86_64.whl (2.5 MB)


In [36]:
%%time
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter('model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

preds = []
labels = []
count = 0
N = 2000 #train.shape[0]
exclude_path = []
for i in tqdm(range(N)):
    frames = load_relevant_data_subset(f'./data/{train.iloc[i].path}')
    output = prediction_fn(inputs=frames)

    sign = np.argmax(output["outputs"])
    topk = np.argsort(output['outputs'])[0][-1:]
    label = label_index[train.iloc[i].sign]
    if label in topk:
        count += 1
    else:
        exclude_path.append(train.iloc[i].path)

count/N


# 114

100%|██████████| 2000/2000 [01:17<00:00, 25.79it/s]

CPU times: user 1min 22s, sys: 2.54 s, total: 1min 24s
Wall time: 1min 17s


0.0065